# Рекомендация тарифов

 - В нашем распоряжении данные о поведении клиентов, которые уже пользуются тарифамиторая выберет подходящий тариф.
 - Данные известные в таблице о использованных в рамках пакета услуг(мегабайты, смс-ки и звонки), данные о пользователе и его тарифе при этом.
 - Создадим модель с максимально большим значением *accuracy* и доведем долю правильных ответов по крайней мере до 0.75.
 - Проверим *accuracy* на тестовой выборке.
 - Оценим модели на адекватность.

## Загрузка файлов

In [1]:
import pandas as pd
import time
import sklearn.metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

## Разбейте данные на выборки

In [4]:
df.head() #изучаем датасет

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
x = df.drop('is_ultra', axis=1)
y = df['is_ultra']
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.4, random_state=12345, stratify=y)
x_valid, x_test, y_valid, y_test = train_test_split(x_valid, y_valid, test_size=0.5, random_state=12345, stratify=y_valid)
#поделила выборки на валидационную, тестовую и тренировочную.


In [6]:
print(x_train.shape) #узнала размеры выборок
print(y_train.shape)
print(y_test.shape)
print(x_test.shape)

(1928, 4)
(1928,)
(643,)
(643, 4)


## Исследование модели

In [7]:
%%time
best_result = 0
best_depth = 0
for depth in range (1, 101):
    model =  DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(x_train, y_train) # обучаем модель на тренировочной выборке
    predictions_valid = model.predict(x_valid) # вычисляем предсказания модели на валидационной выборке
    result = accuracy_score(y_valid, predictions_valid) # определяем качество модели на валидационной выборке
    if result > best_result:
        best_result = result
        best_depth = depth
print("наилучшая глубина", best_depth, "наилучшая точность при этом", best_result)

наилучшая глубина 8 наилучшая точность при этом 0.80248833592535
CPU times: user 1.07 s, sys: 1.03 ms, total: 1.08 s
Wall time: 1.07 s


In [8]:
model = DecisionTreeClassifier(random_state=12345, max_depth = best_depth)
model.fit(x_train, y_train) #подставляем наилучшее значение гиперпараметра в модель
predictions_valid = model.predict(x_valid)
accuracy = accuracy_score(y_valid, predictions_valid)
print("Accuracy наилучшей модели на тестовой выборке:", accuracy)

Accuracy наилучшей модели на тестовой выборке: 0.80248833592535


In [9]:
%%time
best_result_1 = 0
best_depth_1 = 0
best_estimators_1 = 0
for depth in range(5, 13):
    for est in range (1, 101):
        model_1 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_1.fit(x_train, y_train)
        predictions_valid_1 = model_1.predict(x_valid) 
        result = accuracy_score(y_valid, predictions_valid)
        if result > best_result_1:
            best_result_1 = result
            best_estimators_1 = est
            best_depth_1 = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result_1, "Лучшее количество деревьев", 
      best_estimators_1, 'Лучшая глубина дерева:', best_depth_1)

Accuracy наилучшей модели на валидационной выборке: 0.80248833592535 Лучшее количество деревьев 1 Лучшая глубина дерева: 5
CPU times: user 2min 3s, sys: 784 ms, total: 2min 3s
Wall time: 2min 3s


In [10]:
model_1 = RandomForestClassifier(random_state=12345, n_estimators = best_estimators_1, max_depth=best_depth_1)
model_1.fit(x_train, y_train) 
predictions_valid_1 = model_1.predict(x_valid)
accuracy_1 = accuracy_score(y_valid, predictions_valid_1)
print("Accuracy наилучшей модели на тестовой выборке:", accuracy_1)

Accuracy наилучшей модели на тестовой выборке: 0.7713841368584758


## Проверка модели на тестовой выборке

In [11]:
predictions_test = model.predict(x_test)
accuracy = accuracy_score(predictions_test, y_test)
print("Accuracy наилучшей модели на тестовой выборке:", accuracy)

Accuracy наилучшей модели на тестовой выборке: 0.7838258164852255


In [12]:
model_1 = RandomForestClassifier(random_state=12345, n_estimators = best_estimators_1, max_depth=best_depth_1)
model_1.fit(x_test, y_test) # проверяем модель на тренировочной выборке
predictions_test_1= model_1.predict(x_test)
accuracy_1 = accuracy_score(predictions_test_1, y_test)
print("Accuracy наилучшей модели на тестовой выборке:", accuracy_1)

Accuracy наилучшей модели на тестовой выборке: 0.8444790046656299


**Вывод**: для более точных расчетов следует выбрать модель случайного леса, тк на тестовой выборке ее показатели превысили 0.75.

## Проверка моделей на адекватность

In [13]:
predictions_all = model.predict(x) #расчет по первой модели ко всем признакам 
y_pred = predictions_all #правильные ответы модели
y_true = df['is_ultra'] #правильные ответы все
recall = sklearn.metrics.recall_score(y_true, y_pred, pos_label=1)
recall #менее точная оценка

0.5685279187817259

In [14]:
precision = sklearn.metrics.precision_score(y_true, y_pred, pos_label=1)
precision #более точная оценка

0.8247422680412371

In [15]:
predictions_all_1 = model_1.predict(x)
y_pred_1 = predictions_all_1
recall_1 = sklearn.metrics.recall_score(y_true, y_pred_1, pos_label=1)
recall_1

0.4558375634517767

In [16]:
precision_1= sklearn.metrics.precision_score(y_true, y_pred_1, pos_label=1)
precision_1

0.7636054421768708

**Вывод:** Значения параметров recall и precision выше 0.5 у обеих модель, значит можно говорить, что они как минимум точнее случайных. Однако, у второй модели эти параметры незначительно больше, что заставляет нас отдать предпочтения ей, то есть модели случайных лесов.